In [ ]:
import pandas as pd
import pandas_ta as ta
import importlib
import os
import sys
import yfinance as yf

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

Input needed:

1. prices.csv
-date, asset, Close, Expected Return (from AR model), other components needed to run strategies

2. strategy_map.csv
- asset, strategy (e.g., Strategy05), ideal_proportion

Config Example:

config = {
    "starting_cash": 1000000,        # Initial capital
    "buy_pct": 0.05,               # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,        # Minimum 10% cash must be held
    "cash_ceiling_pct": 0.30,      # If cash > 30%, consider buying gold
    "fee": 0.001,                  # Transaction fee (0.1%)
    "tp_pct": 0.10,                # Take-profit threshold (10%) 
    "sl_pct": 0.05                 # Stop-loss threshold (5%)
}

Notes: the tp, sl pct gonna make "dynamic" - different for each asset.

In [ ]:
# Create some sample data to try now
config = {
    "starting_cash": 1000000,  # Initial capital
    "buy_pct": 1,  # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,  # Minimum 10% cash must be held
    "tp_pct": 0.1,  # Take-profit threshold (10%)
    "sl_pct": 0.05,  # Stop-loss threshold (5%)
}

classname_to_filename = {
    "BollingerBandsBreakout": "bollinger_bands_breakout",
    "LarryWilliamsPriceAction": "larry_williams_price_action",
    "MACDBollingerBandsMeanReversion": "macd_bollinger_bands_mean_reversion",
    "MeanReversion": "mean_reversion",
    "MichaelHarrisPriceAction": "michael_harris_price_action",
    "Momentum": "momentum",
    "RSIDivergence": "rsi_divergence",
    "Scalping": "scalping",
    "VolumeSpikeReversal": "volume_spike_reversal",
}

#CHANGE FILE HERE
strategy_map = pd.read_csv("../data/experiments/asset_strategies_2_months_with_tpsl.csv")
strategy_map.head()

In [ ]:
idx= strategy_map.groupby("Asset")["Return [%]"].idxmax()
best = strategy_map.loc[idx]
strategy_map_new = best[["Asset", "Weight", "Strategy", "Return [%]"]]
strategy_map_new = strategy_map_new.set_index("Asset")
strategy_map_new

First function retrieves data from yf 

In [ ]:
def get_asset_df(strategy_map, start, end):
    df_list = []
    for asset_name in strategy_map.index:
        # Retrieve yf finance data
        df = yf.Ticker(asset_name).history(start=start, end=end, actions=False)

        # Align date formatting
        df = df.reset_index()
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.strftime("%Y-%m-%d")

        df["Asset"] = asset_name
        df_list.append(df.reset_index())

    df_all = pd.concat(df_list).set_index(["Date", "Asset"]).sort_index()
    return df_all

In [ ]:
asset_df = get_asset_df(strategy_map=strategy_map_new, start="2023-01-01", end="2025-03-01")
asset_df.head()

Second function implements AR model to derive expected return 

In [ ]:
def compute_returns(asset_df):
    asset_df["Return"] = asset_df.groupby("Asset")["Close"].pct_change()
    return asset_df


asset_df_with_returns = compute_returns(asset_df)

In [ ]:
asset_df_with_returns.head(10)

In [ ]:
def generate_expected_returns(data, ar_results):
    """ Calculate expected returns using fitted AR models """
    new_data = data.reset_index().merge(ar_results, on="Asset", how="left").set_index(["Date", "Asset"])
    new_data["Expected Return"] = new_data["const"] + new_data["Return"] * new_data["ar.L1"]
    return new_data

In [ ]:
ar_model_info = pd.read_csv("../data/processed/ar_model.csv")
ar_model_info

In [ ]:
# Apply AR model
asset_df_with_AR = generate_expected_returns(asset_df_with_returns, ar_model_info)
asset_df_with_AR = asset_df_with_AR.drop(['Best Model','BIC', 'const','ar.L1', 'sigma2'], axis=1)
asset_df_with_AR

In [ ]:
def calculate_technical_indicators(df):
    df = df.copy()

    # Core indicators
    df["EMA_50"] = ta.ema(df["Close"], length=50)
    df["EMA_200"] = ta.ema(df["Close"], length=200)
    df["RSI"] = ta.rsi(df["Close"], length=14)
    df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"], length=7)

    # Bollinger Bands
    bbands = ta.bbands(df["Close"], length=20)
    bbands = bbands.rename(
        columns={
            "BBU_20_2.0": "Upper_Band",
            "BBM_20_2.0": "Middle_Band",
            "BBL_20_2.0": "Lower_Band",
            "BBB_20_2.0": "Band_Width",
            "BBP_20_2.0": "Percent_B",
        }
    )

    # MACD
    macd = ta.macd(df["Close"])
    macd = macd.rename(
        columns={
            "MACD_12_26_9": "MACD",
            "MACDh_12_26_9": "Histogram",
            "MACDs_12_26_9": "Signal",
        }
    )

    # Join indicator DataFrames
    df = df.join([bbands, macd])

    return df

In [ ]:
enriched_df = calculate_technical_indicators(asset_df_with_AR)
enriched_df.head()

Third function to add signal from strategies 

In [ ]:
def load_data_and_apply_strategies(df_all, strategy_map):
    df_list = []

    for asset_name in strategy_map.index:
        # try:
        # Get strategy module + class
        strat_class_name = strategy_map.loc[asset_name, "Strategy"]
        strat_file_name = classname_to_filename[strat_class_name]

        strategy_module = importlib.import_module(
            f"strategies.custom.{strat_file_name}"
        )
        strategy_class = getattr(strategy_module, strat_class_name)
        strategy_instance = strategy_class()

        # Get asset's DataFrame slice
        asset_df = df_all.xs(asset_name, level="Asset").copy()

        # Apply strategy
        asset_df = strategy_instance.generate_signals(asset_df)

        # Restore asset label if removed
        asset_df["Asset"] = asset_name
        df_list.append(asset_df.reset_index())

        # except Exception as e:
        #     print(f"Error processing {asset_name} ({strat_class_name}): {e}")
        #     continue

    # Combine and restore multi-index
    df_with_signals = pd.concat(df_list).set_index(["Date", "Asset"]).sort_index()
    return df_with_signals

In [ ]:
df_with_signals = load_data_and_apply_strategies(enriched_df, strategy_map_new)

In [ ]:
valid_df_with_signals = df_with_signals.loc[df_with_signals.index.get_level_values('Date') >= '2025-01-01']
valid_df_with_signals

In [ ]:
def calculate_fee(amount):
    return max(0.00025 * amount, 0.50)

In [ ]:
#With TP and SL and NO CASH FLOOR
def auto_trade_TP_SL(df, strategy_map, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(["date", "asset"])

    assets = df.index.get_level_values(1).unique()
    print(assets)
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {"units": 0, "entry_price": None} for asset in assets}
    cash = config["starting_cash"]
    trade_log = []
    daily_values = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]["units"] * df.loc[(date, asset), "Close"]
            for asset in assets
            if (date, asset) in df.index
        )

        # Calculate thresholds
        #cash_floor = config["cash_floor_pct"] * total_value
        # cash_ceiling = config['cash_ceiling_pct'] * total_value

        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row["Close"]
            signal = row["TotalSignal"]
            # asset_type = strategy_map.loc[asset, 'type']
            # asset_type = strategy_map.loc[strategy_map['asset'] == asset, 'type'].values[0]

            # Get max allocation for this asset
            ideal_proportion = strategy_map.loc[asset, "Weight"]
            # ideal_proportion = strategy_map.loc[strategy_map['asset'] == asset, 'weight'].values[0]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]["units"] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config["buy_pct"] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # TP/SL check
            
            if portfolio[asset]["units"] > 0:
                entry_price = portfolio[asset]["entry_price"]
                profit_pct = (current_price - entry_price) / entry_price
                if (
                    profit_pct >= config["tp_pct"] or profit_pct <= -config["sl_pct"]
                ):  # IMPLEMENT DYNAMIC TP AND SL
                    if asset.endswith("-USD"):  # Crypto
                        sell_price = current_price * 0.99  # apply 1% spread
                        proceeds = portfolio[asset]["units"] * sell_price
                    else:
                        fee = calculate_fee(portfolio[asset]["units"] * current_price)
                        proceeds = portfolio[asset]["units"] * current_price - fee
                    cash += proceeds
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": (
                                "SELL_TP"
                                if profit_pct >= config["tp_pct"]
                                else "SELL_SL"
                            ),
                            "price": current_price,
                            "units": portfolio[asset]["units"],
                        }
                    )
                    portfolio[asset] = {"units": 0, "entry_price": None}
                    continue  # skip further actions after selling

            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]["units"] > 0:
                if asset.endswith("-USD"):  # Crypto
                    sell_price = current_price * 0.99  # apply 1% spread
                    proceeds = portfolio[asset]["units"] * sell_price
                else:
                    fee = calculate_fee(portfolio[asset]["units"] * current_price)
                    proceeds = portfolio[asset]["units"] * current_price - fee
                cash += proceeds
                trade_log.append(
                    {
                        "asset": asset,
                        "date": date,
                        "action": "SELL_SHORT",
                        "price": current_price,
                        "units": portfolio[asset]["units"],
                    }
                )
                portfolio[asset] = {"units": 0, "entry_price": None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                expected_return = row.get("Expected Return", None)
                baseline_fee_pct = 0.00025
                if expected_return is None or expected_return <= 2 * baseline_fee_pct:
                    continue  # don't buy if expected return is missing or too low

                # Check if enough cash remains after purchase
                if buy_amount > 0:
                    if asset.endswith("-USD"):  # Crypto
                        buy_price = current_price * 1.01  # apply 1% spread
                        new_units = buy_amount / buy_price
                    else:
                        fee = calculate_fee(buy_amount)
                        new_units = (buy_amount - fee) / current_price

                    old_units = portfolio[asset]["units"]
                    old_price = portfolio[asset]["entry_price"]

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price)
                        / total_units
                        if old_units > 0
                        else current_price
                    )  # update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]["units"] = total_units
                    portfolio[asset]["entry_price"] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": "BUY_ADD" if old_units > 0 else "BUY_NEW",
                            "price": current_price,
                            "units": new_units,
                        }
                    )
        eod_value = cash + sum(
            portfolio[asset]['units'] * df.loc[(date, asset), 'Close']
            for asset in assets if (date, asset) in df.index
        )
        # Build daily row with total value and each asset's value
        daily_row = {'date': date, 'portfolio_value': eod_value}
        for asset in assets:
            if (date, asset) in df.index:
                asset_price = df.loc[(date, asset), 'Close']
                asset_units = portfolio[asset]['units']
                daily_row[asset] = asset_units * asset_price
            else:
                daily_row[asset] = 0  # no data that day

        daily_values.append(daily_row)

    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]["units"] * df.loc[(dates[-1], asset), "Close"]
        for asset in assets
        if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config["starting_cash"]

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Add a signed P&L for each trade
        trade_df["signed_value"] = trade_df["price"] * trade_df["units"]
        trade_df["signed_value"] = trade_df.apply(
            lambda row: (
                -row["signed_value"]
                if row["action"].startswith("BUY")
                else row["signed_value"]
            ),
            axis=1,
        )

        # Net P&L by asset
        pnl_per_asset = trade_df.groupby("asset")["signed_value"].sum().sort_values()

        print("\nNet P&L by Asset (lowest to highest):")
        print(pnl_per_asset)

        worst_asset = pnl_per_asset.idxmin()
        worst_loss = pnl_per_asset.min()

        print(
            f"\nAsset that pulled down profit the most: {worst_asset} (${round(worst_loss, 2)})"
        )

    else:
        print("\n(No trades were executed during the period.)")
    
    pd.DataFrame(daily_values).to_csv("../data/misc/daily_portfolio_value.csv", index=False)
    print("Daily portfolio value saved to 'daily_portfolio_value.csv'")

    trade_df.to_csv("../data/misc/trade_log.csv", index=False)
    print("Trade log saved to 'trade_log.csv'")

    return trade_df, final_value

In [ ]:
auto_trade_TP_SL(valid_df_with_signals, strategy_map_new, config)

In [ ]:
#Without TP and SL
def auto_trade_no_TP_SL(df, strategy_map, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(["date", "Asset"])

    assets = df.index.get_level_values(1).unique()
    print(assets)
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {"units": 0, "entry_price": None} for asset in assets}
    cash = config["starting_cash"]
    trade_log = []
    daily_values = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]["units"] * df.loc[(date, asset), "Close"]
            for asset in assets
            if (date, asset) in df.index
        )

        # Calculate thresholds
        #cash_floor = config["cash_floor_pct"] * total_value
        # cash_ceiling = config['cash_ceiling_pct'] * total_value

        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row["Close"]
            signal = row["TotalSignal"]
            # asset_type = strategy_map.loc[asset, 'type']
            # asset_type = strategy_map.loc[strategy_map['asset'] == asset, 'type'].values[0]

            # Get max allocation for this asset
            ideal_proportion = strategy_map.loc[asset, "Weight"]
            # ideal_proportion = strategy_map.loc[strategy_map['asset'] == asset, 'weight'].values[0]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]["units"] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config["buy_pct"] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]["units"] > 0:
                if asset.endswith("-USD"):  # Crypto
                    sell_price = current_price * 0.99  # apply 1% spread
                    proceeds = portfolio[asset]["units"] * sell_price
                else:
                    fee = calculate_fee(portfolio[asset]["units"] * current_price)
                    proceeds = portfolio[asset]["units"] * current_price - fee
                cash += proceeds
                trade_log.append(
                    {
                        "asset": asset,
                        "date": date,
                        "action": "SELL_SHORT",
                        "price": current_price,
                        "units": portfolio[asset]["units"],
                    }
                )
                portfolio[asset] = {"units": 0, "entry_price": None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                expected_return = row.get("Expected Return", None)
                baseline_fee_pct = 0.00025
                if expected_return is None or expected_return <= 2 * baseline_fee_pct:
                    continue  # don't buy if expected return is missing or too low

                # Check if enough cash remains after purchase
                if buy_amount > 0:
                    if asset.endswith("-USD"):  # Crypto
                        buy_price = current_price * 1.01  # apply 1% spread
                        new_units = buy_amount / buy_price
                    else:
                        fee = calculate_fee(buy_amount)
                        new_units = (buy_amount - fee) / current_price

                    old_units = portfolio[asset]["units"]
                    old_price = portfolio[asset]["entry_price"]

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price)
                        / total_units
                        if old_units > 0
                        else current_price
                    )  # update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]["units"] = total_units
                    portfolio[asset]["entry_price"] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": "BUY_ADD" if old_units > 0 else "BUY_NEW",
                            "price": current_price,
                            "units": new_units,
                        }
                    )
        eod_value = cash + sum(
            portfolio[asset]['units'] * df.loc[(date, asset), 'Close']
            for asset in assets if (date, asset) in df.index
        )
        # Build daily row with total value and each asset's value
        daily_row = {'date': date, 'portfolio_value': eod_value}
        for asset in assets:
            if (date, asset) in df.index:
                asset_price = df.loc[(date, asset), 'Close']
                asset_units = portfolio[asset]['units']
                daily_row[asset] = asset_units * asset_price
            else:
                daily_row[asset] = 0  # no data that day

        daily_values.append(daily_row)

    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]["units"] * df.loc[(dates[-1], asset), "Close"]
        for asset in assets
        if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config["starting_cash"]

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Add a signed P&L for each trade
        trade_df["signed_value"] = trade_df["price"] * trade_df["units"]
        trade_df["signed_value"] = trade_df.apply(
            lambda row: (
                -row["signed_value"]
                if row["action"].startswith("BUY")
                else row["signed_value"]
            ),
            axis=1,
        )

        # Net P&L by asset
        pnl_per_asset = trade_df.groupby("asset")["signed_value"].sum().sort_values()

        print("\nNet P&L by Asset (lowest to highest):")
        print(pnl_per_asset)

        worst_asset = pnl_per_asset.idxmin()
        worst_loss = pnl_per_asset.min()

        print(
            f"\nAsset that pulled down profit the most: {worst_asset} (${round(worst_loss, 2)})"
        )

    else:
        print("\n(No trades were executed during the period.)")
    
    pd.DataFrame(daily_values).to_csv("../data/misc/daily_portfolio_value_no_TP_SL.csv", index=False)
    print("Daily portfolio value saved to 'daily_portfolio_value_no_TP_SL.csv'")

    trade_df.to_csv("../data/misc/trade_log_no_TP_SL.csv", index=False)
    print("Trade log saved to 'trade_log_no_TP_SL.csv'")

    return trade_df, final_value

In [ ]:
auto_trade_no_TP_SL(valid_df_with_signals, strategy_map_new, config)